# Implementación de *Simulated Annealing* para el problema del agente viajero

**Importar las librerías necesarias para poder trabajar con los datos**

In [1]:
import pandas as pd
import numpy as np
from numpyencoder import NumpyEncoder
import matplotlib.pyplot as plt
import json

**Leer los datos (la matriz de distancias) y crear un DataFrame a partir de ellos**

In [2]:
CITIES = np.empty((128, 3))

with open('coordinates.txt', 'r') as f:
    for index, line in enumerate(f.readlines()):
        line = line.strip().split()
        CITIES[index] = [index, float(line[0]), float(line[1])]

DATA = pd.read_csv('cities.csv', header=None)

# Vamos a imprimir los primeros 5 para asegurarnos de que todo está bien
DATA.head(5)

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0,966,1513,2964,1149,927,1611,1510,390,466,...,1018,168,565,1700,1636,2019,1458,1564,2871,348
1,966,0,2410,1520,1817,729,686,290,1823,168,...,548,1127,432,2265,558,571,943,198,1917,2541
2,1513,2410,0,604,481,2742,1833,826,214,1618,...,752,486,590,2132,1095,2154,1211,2217,2673,1570
3,2964,1520,604,0,595,1289,1446,466,1139,430,...,1861,861,499,1408,986,2719,1437,769,1038,2343
4,1149,1817,481,595,0,494,550,2641,765,934,...,216,1994,324,2187,260,2586,1974,2352,2243,691


**Hacer un primer camino aleatorio**

In [3]:
PATH = np.random.permutation(CITIES)

**Definir la función del costo del camino actual**

In [4]:
def cost(path: np.array) -> float:
    """Calcula el costo de la ruta elegida al momento

    Args:
        path (np.array): Array que contiene la ruta que se debe de seguir al momento

    Returns:
        float: El costo de la ruta
    """
    global DATA
    
    cost = 0
    
    # Calcular el costo del camino
    for i in range(len(path) - 1):
        cost += DATA.iloc[int(path[i, 0]), int(path[i + 1, 0])]
        
    # Sumar el costo de volver al principio
    cost += DATA.iloc[int(path[-1, 0]), int(path[0, 0])]
    return cost

**Definir la función de obtener un nuevo camino 'vecino' del actual**

In [5]:
def generate_path(path: np.array) -> np.array:
    """Genera una ruta tomando dos elementos aleatorios de la ruta, y haciendo un 'reverse' de todos los elementos 
        dentro del array formado por estas dos ciudades como extremos

    Args:
        path (np.array): La ruta actual

    Returns:
        np.array: La nueva ruta
    """
    # Tomar dos elementos aleatorios
    i, j = np.random.choice([i for i in range(0, 128)], size=2, replace=False)
    
    # Hacer el swap
    new_path = np.copy(path)
    new_path[i : j] = path[i : j][::-1]
    
    return new_path

## Función que usa todas las funciones anteriores para generar la mejor ruta

In [6]:

def simulated_annealing(path: np.array, t: float, l: int, max_loops: int = 2000000, t_update: float = 0.999, save_graph_values: bool = True, global_min: float = None, seed: int = 0) -> list:
    """Función que implementa el algoritmo de Simulated Annealing

    Args:
        path (np.array): La ruta actual
        t (float): Temperatura
        l (int): Cada cuantas iteraciones se debe de actualizar la temperatura
        max_loops (int, optional): Cantidad máxima de iteraciones. Defaults to 1000000.
        t_update (float, optional): Factor de actualización de la temperatura. Defaults to 0.99.
        save_graph_values (bool, optional): Si se debe de guardar los valores de la función de costo para graficar. Defaults to True.
        global_min (float, optional): Valor mínimo global. Defaults to None.
        seed (int, optional): Semilla para el random. Defaults to 0.

    Returns:
        np.array: La nueva ruta
    """
    np.random.seed(seed)
    
    first_t = t
    diff = 0
    k = 0
    cost_path = cost(path)
    super_min_cost = cost_path
    results = []
    t_values = []
    information = {
        'T': t,
        'L': l,
        't_update': t_update,
        'max_loops': max_loops,
        'paths': [
            path
        ]
    }

    while k < max_loops:
        # Generar una nueva ruta
        new_path = generate_path(path)

        # Calcular la diferencia de costos
        cost_new_path = cost(new_path)
        diff = cost_new_path - cost_path
        
        # Calcular el minimo costo que se ha tenido en todas las corridas y si es menor que el actual actualizar la información
        if cost_new_path < super_min_cost:
            super_min_cost = cost_new_path
            information['paths'].append(new_path)

        # Si la diferencia es menor que 0, entonces la nueva ruta es mejor, por lo que la actual se debe de actualizar
        if diff < 0:
            path = new_path
            cost_path = cost_new_path

        # Si la diferencia es mayor que 0, entonces la nueva ruta es peor, por lo que se debe de probar si debe de ser aceptada
        elif np.random.rand() < np.exp(-diff / t):
            path = new_path
            cost_path = cost_new_path

        # Escribir el costo dentro del array de resultados y T para graficar
        results.append(cost_path)
        t_values.append(t)

        # Aumentar K y cada L pasos disminuir T
        k += 1
        if k % l == 0:
            t *= t_update
            
            
    # Guardar la información dentro de un archivo
    if save_graph_values and super_min_cost < global_min:
        with open(f'files/SA_t={first_t}_l={l}_tupdate={t_update}_seed={seed}_maxloops={max_loops}_result={results[-1]}_minimum={super_min_cost}.json', 'w') as file:
            json.dump(information, file, indent=0, sort_keys=True,separators=(',', ':'), ensure_ascii=False, cls=NumpyEncoder)
    
    return results, t_values, super_min_cost
    
    

**Ejecutar el programa**

En la siguiente celda es donde vamos a ejecutar todo el programa. El programa se ejecutó durante toda una noche para poder encontrar la mejor solución posible. Para poder correr el programa se tiene que dejar la variable ```execute=True```.

**NO CORRAS ESTA CELDA PARA VISUALIZAR EL CÓDIGO**
- Es un loop infinito
- Tarda aprox. 7min por iteración
- Se puede visualizar el notebook sin correr la celda con los valores ya guardados

**SOLO CORRELA SI LO QUE QUIERES ES HACER TU MISMO LA SIMULACIÓN**

In [7]:
execute = True
save_graph = True
graph_t = False
global_min = cost(PATH)
counter = 0


if execute:
    while True:
        counter += 1
        
        t = np.random.normal(loc=2500.0, scale=500, size=1)[0]
        l = np.random.normal(loc=100.0, scale=20,size=1)[0]
        t_update = np.random.normal(loc=0.997, scale=0.001,size=1)[0]
        max_loops = 200000
        seed = np.random.randint(0, 1000000000)
    
        # run algorithm
        results, t_values, min_cost = simulated_annealing(path=PATH, t=t, l=int(l), max_loops=max_loops, t_update=t_update, save_graph_values=True, global_min=global_min, seed=seed)
    
        # graph results
        if save_graph and min_cost < global_min:
            global_min = min_cost
            
            plt.figure(figsize=(8,6), facecolor='w')
            plt.plot(range(len(results)), results)
            plt.title(f'Cost of Simulated Annealing. min={min_cost}')
            plt.xlabel('k iterations')
            plt.ylabel('Cost')
    
            # save plot on disk
            plt.savefig(f'./plots/SA_t={t}_l={l}_tupdate={t_update}_seed={seed}_maxloops={max_loops}_result={results[-1]}_minimum={min_cost}.png')
            plt.clf()
            
            # print results on screen
            print(f"{counter}. cost: {min_cost}")
    
            ########################
            if graph_t:
                # graph T values
                plt.plot(range(len(t_values)), t_values)
                plt.title(f'Value of T')
                plt.xlabel('k iterations')
                plt.ylabel('T')
        
                # save plot on disk
                plt.savefig(f'./plots/T_t={t}_l={l}_tupdate={t_update}_seed={seed}_maxloops={max_loops}_result={results[-1]}_minimum={min_cost}.png')
                plt.clf()
        
        print(f"Finished with min_cost: {min_cost}")

1. cost: 24257
Finished with min_cost: 24257
Finished with min_cost: 24975
3. cost: 23573
Finished with min_cost: 23573
Finished with min_cost: 26322
Finished with min_cost: 26006
Finished with min_cost: 25375
Finished with min_cost: 24598
Finished with min_cost: 25792
Finished with min_cost: 24966
Finished with min_cost: 24949
Finished with min_cost: 25689
Finished with min_cost: 26559
Finished with min_cost: 24316
Finished with min_cost: 24509
Finished with min_cost: 25748
Finished with min_cost: 25443
Finished with min_cost: 23868
Finished with min_cost: 24055
Finished with min_cost: 28096
Finished with min_cost: 24400
Finished with min_cost: 25013
Finished with min_cost: 25810
Finished with min_cost: 26069
Finished with min_cost: 24572
Finished with min_cost: 23623
Finished with min_cost: 24389
Finished with min_cost: 27965
Finished with min_cost: 25305
Finished with min_cost: 61822
Finished with min_cost: 26641
Finished with min_cost: 24623
Finished with min_cost: 25781
Finished w

Después de varias corridas, nos damos cuenta de que el mejor resultado es el siguiente: ```135,434``` km.

Sin embargo, se logró cuando todavía no guardaba ni el *seed* ni la información de las corridas para poder efectuar la animación, es por eso que la animación se hará con otra corrida. Sin embargo, las variables fueron las siguientes en la mejor corrida: ```(T=3000, L=100, t_update=0.997)```, y en esa corrida si se guardó la gráfica que es la siguiente:

<img src="plots/SA_t=3000_l=100_tupdate=0.997_d=135434.png"/>

La animación de la corrida se hará con los datos de una corrida con los mismos valores ```(T=3000, L=100, t_update=0.997, seed=698133137)``` pero que tuvo un costo de ```136,570``` km, solo ```1,136``` km más que la mejor (un incremento del 0.8% que puede despreciarse). La gráfica del costo de dicha corrida es la siguiente:

<img src="plots/SA_t=3000_l=100_tupdate=0.997_s=698133137_d=136570_min=136570.png"/>

Además, podemos graficar cuál es el decremento de T conforme avanza el tiempo (conforme k va creciento). Lo podemos ver en la siguiente gráfica (valores ```T=2000, L=100, t_update=0.998, max_loops=100000```):

<img src="plots/T_t=2000_l=100_tupdate=0.998_maxloops=100000.png"/>




## Animación

**Primero vamos a graficar todas las coordenadas**

In [ ]:
COORDS = np.empty((128, 2))

with open('coordinates.txt', 'r') as f:
    for index, line in enumerate(f.readlines()):
        line = line.strip().split()
        COORDS[index] = [float(line[0]), float(line[1])]

In [ ]:
plt.figure(figsize=(8,6), facecolor='w')

plt.plot(COORDS.T[0], COORDS.T[1], 'o', c='black')
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Cities")

plt.savefig(f'./map.png')
plt.show()

**Ahora vamos a crear todos los plots y guardarlos como imagenes para poderlas juntar después como una visualización de animación**

```
# Declarar la variable que va a guardar toda la información
information = {}

# Leer el archivo que resultó con toda la información de la corrida
with open('./files/SA_t=3000_l=100_tupdate=0.997_d=39839_02.json', 'r') as file:
    information = json.load(file)
    
paths = np.array(information['paths'])
for i in range(len(paths)):
    plt.figure(figsize=(8,6), facecolor='w')
    plt.plot(paths[i, :, 1], paths[i, :, 2], '-o')
    plt.title(f'step: {i}.  Cost: {cost(paths[i, :, :])}')
    plt.xlabel('X')
    plt.ylabel('Y')

    # save plot on disk
    # plt.savefig(f'./plots/animation/{i}.png')
    # plt.show()
    # plt.clf()